# COMP 424 Assignment 1

## Implement Hillclimbing

In [53]:
import numpy as np
import random
import pandas as pd

def find_max(x, y, function, step = 0.01, lower_bound = 0, upper_bound = 10):
    neighbors = [x - step, y - step, x - step, y + step, x + step, y - step, x + step, y + step, x, y - step, x, y + step, x - step, y, x + step, y]
    maxi = function(x , y)
    coordinates = [x , y]
    for index in range(0, len(neighbors) - 1, 2): 
        result = function(neighbors[index], neighbors[index + 1])
        if result > maxi and not(neighbors[index] < lower_bound or neighbors[index] > upper_bound or neighbors[index + 1] < lower_bound or neighbors[index + 1] > upper_bound):
            maxi = result
            coordinates = [neighbors[index], neighbors[index + 1]]
    return maxi, coordinates

        

## Initialize Functions

In [2]:
def function_one(x, y):
    return np.sin(x / 2) + np.cos(y / 2)

In [51]:
def algorithm(init_x, init_y, function, step_size):
    result_old = find_max(init_x, init_y, function, step_size)
    iteration = 1
    while True:
        result = find_max(result_old[1][0], result_old[1][1], function, step_size)
        if result[1][0] == result_old[1][0] and result[1][1] == result_old[1][1]:
            return {'init_x' : init_x,'init_y': init_y,'max': result[0],'end_x': result[1][0], 'end_y': result[1][1], 'iteration': iteration}
        result_old = result
        iteration += 1
    return None

## Run Algorithm

In [56]:
step = 0.01
results = []
for i in range(0, 100): 
    init_x = random.randint(0 , 10)
    init_y = random.randint(0 , 10)
    result = algorithm(init_x, init_y, function_one, step)
    results.append(result)
frame = pd.DataFrame(data = results)

In [59]:
frame

,end_x,end_y,init_x,init_y,iteration,max
0,3.14,1.000000e+01,2,7,300,1.283662
1,3.14,1.000000e+01,0,7,314,1.283662
2,3.14,8.362408e-14,0,6,600,2.000000
3,3.14,1.000000e-02,1,2,214,1.999987
4,3.14,1.000000e+01,2,8,200,1.283662
5,3.14,4.099152e-14,1,4,400,2.000000
6,3.14,1.000000e+01,4,8,200,1.283662
7,3.14,8.362408e-14,2,6,600,2.000000
8,3.14,1.000000e+01,7,9,386,1.283662
9,3.14,1.967523e-14,4,3,300,2.000000
